In [15]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [17]:
%run -i ../../zkstats/core.py

In [18]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [19]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [20]:
scales = [6]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [21]:
# Verifier/ data consumer side:
# since have division, scale must be very LARGE, cant use just 0
# So, we allow error rate to be 10% instead of 1%
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x = data[0]
    return s.harmonic_mean(x)

error = 0.1
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/Users/mhchia/projects/work/pse/zk-stats-lib/zkstats/computation.py:166: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/mhchia/Library/Caches/pypoetry/virtualenvs/zkstats-brXmXluj-py3.12/lib/python3.12/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))


In [22]:
# prover calculates settings, send to verifier
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [6]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":6,"param_scale":6,"scale_rebase_multiplier":10,"lookup_range":[-184392,11264],"logrows":18,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":912,"total_const_size":4,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,6],"model_input_scales":[6],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Recip":{"scale":4096.0}},{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [23]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0


==== setting up ezkl ====


spawning module 2
spawning module 0
spawning module 2


Time setup: 46.1549072265625 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 45.21875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[11712583131456230033, 7998639690917952521, 17628064685104548320, 3364894255067083796], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [10409296726488761395, 15557203226960814343, 16692342436085548322, 2420882828764455196]]], 'proof': '1cf8b61368ed906916c88eebd055a9f53e3357380cb2215fcf3d5bcfb331f1d4288a00110e17f3fb3efa02da9adf4a7067c83a634cc53044bdf187a952a89cf10905dd7331b4578968d113dd68012c32740dab9c22a2e33a79e6d574152fb44920e26fe4d0a674a1927d60dddeabec7f3d88f254492fa1151e6ffc16a3217a0d1a29386288bfcf7f8c2a5c89de8e2132413a2bfbbfc68b5cbe2dc27ac23619d427b80400e49a23bcf24f04c801028894b9da35845e8ede0726545128f7849d6200dceb7c7da486945aed19fae35e8d933d6772958ade9e01631ab53607541b662cc182f21071264786b7f915930adf3a14486e5d69a733b4927e773c4e701d801db991c5fc8a4bd1d6aad578927fa03af55829de211da849b908bcc268d4b8dc144dd4d0a03ff24ad3590c97b7e9e578528e600f18f036301b6f2deca397e0ae27366ce2fbd2c81db0d6675d4ae59c01942e8563655ee034cafe367e7637

In [24]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 45.21875
